# Chat by using ollama api

In [ ]:
import requests
from bs4 import BeautifulSoup
from IPython.display import HTML, display, Markdown

OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}
MODEL = "llama3.2"

In [ ]:
# create a system message list
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Describe a full cycle of the Recruitment process"},
]

# create a request payload
payload = {
        "model": MODEL,
        "messages": messages,
        "stream": False
    }

In [ ]:
response = requests.post(OLLAMA_API, json=payload, headers=HEADERS)
print(response.json()['message']['content'])

# Using Python Ollama package

In [ ]:
import ollama

response = ollama.chat(model=MODEL, messages=messages)
print(response['message']['content'])

# Summarize the web page


In [6]:
# A class to represent a Webpage Content
import requests
from bs4 import BeautifulSoup

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [17]:
webcontent = Website("https://ollama.com/blog/functions-as-tools")
print(webcontent.title)
# print(webcontent.text)

Ollama Python library 0.4 with function calling improvements · Ollama Blog


In [11]:
# Define our system prompt - you can experiment with this later, changing the last sentence to 'Respond in markdown in Spanish."

system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}. \
    The contents of this website is as follows; \
    Please provide a short summary of this website in markdown. \
    If it includes news or announcements, then summarize these too.\n\n"
    
    user_prompt += website.text
    return user_prompt

def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]


In [18]:
import ollama

MODEL = "llama3.2"
ed = Website("https://hbr.org/2023/09/how-the-best-chief-data-officers-create-value")

response = ollama.chat(model=MODEL, messages=messages_for(ed))
print(response['message']['content'])

# How the Best Chief Data Officers Create Value

## Summary

The majority of chief data officers (CDOs) fail to value and price the business outcomes created by their data and analytics capabilities, leading to short tenures. However, based on 17 in-depth interviews with CDOs who are considered at the frontier of the role, researchers Suraj Srinivasan and Robin Seibert identified four spheres of influence where CDOs can create value: 

1. **Strategic Leadership**: Emphasizing alignment with business goals and establishing clear metrics for success.
2. **Data Governance**: Developing data management capabilities, ensuring data quality, and implementing effective data governance frameworks.
3. **Data Innovation**: Fostering a culture of innovation, leveraging AI and machine learning to drive insights, and integrating data into decision-making processes.
4. **Organizational Alignment**: Ensuring that the organization is equipped to harness the power of data analytics capabilities.

By foc

In [4]:
import requests
from bs4 import BeautifulSoup
import ollama
import json
from langchain_ollama import ChatOllama

MODEL = "llama3.2"

class llms:
    def __init__(self, model, company, website, url):
        self.model = MODEL
        self.website = website
        self.company = company
        self.url = url
    
        self.llm = ChatOllama(
            model = self.model,
            temperature = 0.8,
            num_predict = 256,
            # other params ...
            )
    
    def get_relevant_links(self):
        system_prompt = """You are provided with a list of links found on a webpage.\n \
        You should only include links that are relevant to the company brochure,\n \
        such as links to an About page, or a Company page, or products, or solution pages.\n
        You should respond in JSON as the example below. I don't need you give any explanation, just a JSON response.\n:
            "links": [
                {"type": "about page", "url": "https://full.url/goes/here/about"},
                {"type": "careers page": "url": "https://another.full.url/careers"}
            ]
        """
    
        user_prompt = f"""Here is the company website of {self.url}.
            I will provide you with a list of links found on the website as below: {self.website.get_links(self.url)} \n
            Please decide which of these are relevant web links for a company brochure, respond with the full https URL in JSON format.
            Step by step review the solutions or products and include them the relevant links.
            Do not include Terms of Service, Privacy, email links."""

        payload = [
                ("system", system_prompt),
                ("human", user_prompt),
            ]

        response = self.llm.invoke(input=payload)
        # print(response)
        # result = response["content"]
        return response

    def get_all_details(self):
        relevant_links = self.get_relevant_links()
        # print(relevant_links)

        details = []
        for link in relevant_links["links"]:
            content = self.website.get_content(link["url"])
            details.append({"type": link["type"], "content": content})
        return details

    def get_brochure_user_prompt(company_name):
        user_prompt = f"""You are looking at a company called: {self.company_name}\n
        Use the below information to build a short brochure of the company in markdown.\n
        The company's website is: {self.website.url}\n
        The wesite's contents are: {self.get_all_details()}"""
        return user_prompt


class Website:
    def __init__(self):
        self.headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
            # Add more headers as needed
        }

    def get_raw_html(self, url):
        response = requests.get(url, headers=self.headers)
        body = response.content
        soup = BeautifulSoup(body, 'html.parser')
        return soup

    def get_title(self, url):
        soup = self.get_raw_html(url)
        return soup.title.string if soup.title else "No title found"

    def get_content(self, url):
        soup = self.get_raw_html(url)
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            text = soup.body.get_text(separator="\n", strip=True)
        else:
            text = ""
        return text

    def get_links(self, url):
        soup = self.get_raw_html(url)
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]
        return f"\n".join(self.links) + "\n\n"

In [5]:
url = "https://python.langchain.com/api_reference/ollama/chat_models/langchain_ollama.chat_models.ChatOllama.html"
website_crape = Website()
print(website_crape.get_title(url))
# print(website_crape.get_content(url))
print(website_crape.get_links(url))

ChatOllama — 🦜🔗 LangChain  documentation
#main-content
../../index.html
../../reference.html
https://python.langchain.com/
https://github.com/langchain-ai/langchain
https://twitter.com/langchainai
../../reference.html
https://python.langchain.com/
https://github.com/langchain-ai/langchain
https://twitter.com/langchainai
../../core/index.html
../../langchain/index.html
../../text_splitters/index.html
../../community/index.html
../../experimental/index.html
../../ai21/index.html
../../anthropic/index.html
../../astradb/index.html
../../aws/index.html
../../azure_dynamic_sessions/index.html
../../box/index.html
../../cerebras/index.html
../../chroma/index.html
../../cohere/index.html
../../couchbase/index.html
../../databricks/index.html
../../elasticsearch/index.html
../../exa/index.html
../../fireworks/index.html
../../google_community/index.html
../../google_genai/index.html
../../google_vertexai/index.html
../../groq/index.html
../../huggingface/index.html
../../ibm/index.html
../../m

In [6]:
llms_instance = llms(model=MODEL, company='Langchain',website=website_crape, url=url)
response = llms_instance.get_relevant_links()


In [7]:
rs = response.content
rs

'Here is the list of relevant web links in JSON format:\n\n```\n{\n  "links": [\n    {\n      "title": "Chat OLLAMA Documentation",\n      "url": "https://ollama.readthedocs.io/en/latest/index.html"\n    },\n    {\n      "title": "OLLA-Ma - AI-powered Conversational Interface",\n      "url": "https://ollama.readthedocs.io/en/latest/ollematools.html"\n    },\n    {\n      "title": "Chat OLLAMA Model Hub",\n      "url": "https://huggingface.co/models?query=OLLAMA"\n    },\n    {\n      "title": "OLLA-Ma GitHub Repository",\n      "url": "https://github.com/ollama/ollematools"\n    }\n  ]\n}\n```\n\nThese links are relevant to the Chat OLLAMA product and its related solutions:\n\n* The documentation link provides an overview of the chat interface and its features.\n* The OLLA-Ma link explains the AI-powered conversational interface that powers the chat interface.\n* The model hub link allows users to explore and access pre-trained models for the chat interface.\n* The GitHub repository li